In [7]:
!pip install scikit-learn

  Using cached joblib-1.4.2-py3-none-any.whl.metadata (5.4 kB)
  Using cached threadpoolctl-3.5.0-py3-none-any.whl.metadata (13 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.4/13.4 MB 505.4 kB/s eta 0:00:0000:0100:02
Using cached joblib-1.4.2-py3-none-any.whl (301 kB)
Using cached threadpoolctl-3.5.0-py3-none-any.whl (18 kB)


In [8]:
import pandas as pd
import numpy as np
from collections import defaultdict
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# Load data from CSV
data = pd.read_csv('spam_ham_dataset.csv')  # Replace with your file path
data.columns = ['text', 'label']  # Ensure column names are correct

# Split data into training and test sets
train_data, test_data = train_test_split(data, test_size=0.3, random_state=42)

# Vocabulary and label counts
vocabulary = set()
spam_count = 0
ham_count = 0
word_counts = {'spam': defaultdict(int), 'ham': defaultdict(int)}

# Process training data
for _, row in train_data.iterrows():
    text, label = row['text'], row['label']
    words = text.split()
    vocabulary.update(words)
    if label == 1:
        spam_count += 1
        for word in words:
            word_counts['spam'][word] += 1
    else:
        ham_count += 1
        for word in words:
            word_counts['ham'][word] += 1

# Prior probabilities
P_spam = spam_count / len(train_data)
P_ham = ham_count / len(train_data)

# Total words and vocabulary size
total_spam_words = sum(word_counts['spam'].values())
total_ham_words = sum(word_counts['ham'].values())
vocab_size = len(vocabulary)

# Likelihoods with Laplace smoothing
likelihoods = {
    'spam': {word: (word_counts['spam'][word] + 1) / (total_spam_words + vocab_size) for word in vocabulary},
    'ham': {word: (word_counts['ham'][word] + 1) / (total_ham_words + vocab_size) for word in vocabulary},
}

# Posterior Probability Function
def classify_text(text):
    words = text.split()
    spam_prob = np.log(P_spam)  # Use log to avoid underflow
    ham_prob = np.log(P_ham)

    for word in words:
        if word in likelihoods['spam']:
            spam_prob += np.log(likelihoods['spam'][word])
            ham_prob += np.log(likelihoods['ham'][word])

    return 1 if spam_prob > ham_prob else 0

# Evaluate on the test set
y_true = test_data['label'].tolist()
y_pred = [classify_text(text) for text in test_data['text']]

# Calculate accuracy
accuracy = accuracy_score(y_true, y_pred)
print(f"Accuracy: {accuracy * 100:.2f}%")


ValueError: Length mismatch: Expected axis has 4 elements, new values have 2 elements